In [36]:
# Install required libraries
!python -m ensurepip --default-pip
!python -m pip --version
!python -m pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib markdown

Looking in links: c:\Users\leoch\AppData\Local\Temp\tmpj6ecj4ta
pip 24.2 from c:\Users\leoch\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip (python 3.12)



ERROR: Could not find a version that satisfies the requirement google-colab (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for google-colab


In [35]:
# Import necessary libraries
import os
import markdown
from google.colab import auth
from googleapiclient.discovery import build
from google_auth_httplib2 import AuthorizedHttp
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload
from io import BytesI

print("All libraries imported successfully!")

ModuleNotFoundError: No module named 'google.colab'

In [29]:
# Step 1: Authenticate and create a Google Docs API service
def authenticate_google_docs():
    # Authenticate and create the Docs API service
    auth.authenticate_user()
    credentials = None
    if os.path.exists('token.json'):
        credentials = google.auth.load_credentials_from_file('token.json')[0]

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', ['https://www.googleapis.com/auth/documents']
            )
            credentials = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(credentials.to_json())

    docs_service = build('docs', 'v1', credentials=credentials)
    return docs_service

In [30]:
# Step 2: Convert markdown to Google Docs formatted text
def markdown_to_google_docs(md_text):
    # Convert markdown to HTML
    html = markdown.markdown(md_text)

    # Create the Google Doc content list
    doc_content = []
    
    # Main title - Heading 1
    doc_content.append({
        'insertText': {
            'location': {'index': 1},
            'text': "Product Team Sync\n",
        }
    })

    # Process sections and subsections
    lines = md_text.split('\n')
    for line in lines:
        if line.startswith("## "):  # Heading 2
            doc_content.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': line[3:] + '\n',
                }
            })
        elif line.startswith("### "):  # Heading 3
            doc_content.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': line[4:] + '\n',
                }
            })
        elif line.startswith("- [ ]"):  # Checkbox item
            doc_content.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': line[6:] + '\n',
                }
            })
        else:
            doc_content.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': line + '\n',
                }
            })

    return doc_content

In [31]:
# Step 3: Create Google Doc from markdown content
def create_google_doc(md_text):
    docs_service = authenticate_google_docs()

    # Create a new Google Doc
    document = docs_service.documents().create().execute()
    doc_id = document['documentId']

    # Convert markdown to formatted Google Docs content
    doc_content = markdown_to_google_docs(md_text)

    # Insert formatted content into the Google Doc
    for content in doc_content:
        docs_service.documents().batchUpdate(
            documentId=doc_id,
            body={'requests': [content]}
        ).execute()

    return doc_id

In [32]:
# Step 4: Generate the Google Doc from the provided markdown meeting notes
md_text = '''
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
'''

# Run the function to create a Google Doc
doc_id = create_google_doc(md_text)
print(f"Google Doc created successfully with ID: {doc_id}")


NameError: name 'auth' is not defined